# 0. 패키지 불러오기

In [ ]:
!pip install catboost
!pip install category_encoders
!pip install optuna
!pip install kmodes

     |████████████████████████████████| 76.1 MB 1.2 MB/s 
     |████████████████████████████████| 82 kB 383 kB/s 
     |████████████████████████████████| 308 kB 4.3 MB/s 
     |████████████████████████████████| 80 kB 9.4 MB/s 
     |████████████████████████████████| 209 kB 62.0 MB/s 
     |████████████████████████████████| 75 kB 4.7 MB/s 
     |████████████████████████████████| 49 kB 5.1 MB/s 
     |████████████████████████████████| 149 kB 52.1 MB/s 
     |████████████████████████████████| 112 kB 49.8 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=9fe22ce2523254e73739ae12ffdb1efda88625ab0034a8fae7c8750aad520a2a
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm # matplotlib의 font_manager에서 폰트설정을 변경하여 matplotlib에서도 한글 출력을 가능하게 한다.
import seaborn as sns

from sklearn.ensemble import * # ExtraTreesRegressor, RandomForestClassifier
from sklearn.model_selection import * # KFold, cross_val_score, train_test_split
from sklearn.metrics import *  # make_scorer : MSE 대신 사용자가 정의한 손실함수를 사용하고 싶을 때
from sklearn.preprocessing import * # LabelEncoder, StandardScaler
from sklearn.svm import * # SVC, SVR
from sklearn.inspection import *
from sklearn.linear_model import * # LogisticRegression, RANSACRegressor, Ridge, Lasso, ElasticNet
from sklearn.decomposition import * # PCA

from category_encoders.ordinal import OrdinalEncoder
from catboost import CatBoostClassifier, Pool
from sklearn.cluster import KMeans
from kmodes.kmodes import KModes
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from pathlib import Path

from lightgbm import *
import lightgbm as lgb
import xgboost as xgb
from datetime import datetime
import calendar
import warnings
warnings.filterwarnings(action = 'ignore') # 경고메세지를 무시하거나 숨긴다.

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 1. 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData"

 cat_part_test.csv    lgbm_val.csv		   tab_test.csv
 cat_test.csv	      참가자_제공_레이아웃.pdf	   tab_val.csv
 cat_val.csv	      rf_part_test.csv		  'test2_수정 전.csv'
 속성_D_코드.csv      rf_test.csv		   test2.csv
 etc_part_test.csv    rf_val.csv		   test.csv
 etc_test.csv	      sample_submission.csv	  'train2_수정 전.csv'
 etc_val.csv	      submission_cat2.csv	   train2.csv
 속성_H_코드.csv      submission_cat.csv	   train.csv
 속성_L_코드.csv      submission_lgbm.csv	   xgb_part_test.csv
 lgbm_part_test.csv   submission_stk_lgbm_wj.csv   xgb_test.csv
 lgbm_test.csv	      tab_part_test.csv		   xgb_val.csv


In [ ]:
Rdata_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/train2.csv')
Rdata_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/test2.csv')
Rdata_D = pd.read_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/속성_D_코드.csv', index_col = 0).T.to_dict()
Rdata_H = pd.read_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/속성_H_코드.csv', index_col = 0).T.to_dict()
Rdata_L = pd.read_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/속성_L_코드.csv', index_col = 0).T.to_dict()
print(Rdata_train.shape)
print(Rdata_train.size)
print(Rdata_test.shape)
print(Rdata_test.size)
Rdata_train.head()

(501951, 97)
48689247
(46404, 96)
4454784


,Unnamed: 0,id,person_attribute_a,person_attribute_a_under,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_f,person_prefer_g,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_under,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,target,person_prefer_d_1_세,person_prefer_d_1_소,person_prefer_d_1_중,person_prefer_d_1_대,person_prefer_d_2_세,person_prefer_d_2_소,person_prefer_d_2_중,person_prefer_d_2_대,person_prefer_d_3_세,person_prefer_d_3_소,person_prefer_d_3_중,...,d1_la_match_yn,d1_mi_match_yn,d1_sm_match_yn,d1_sub_match_yn,d1_all_match_yn,d2_la_match_yn,d2_mi_match_yn,d2_sm_match_yn,d2_sub_match_yn,d2_all_match_yn,d3_la_match_yn,d3_mi_match_yn,d3_sm_match_yn,d3_sub_match_yn,d3_all_match_yn,h1_la_match_yn,h1_mi_match_yn,h1_all_match_yn,h2_la_match_yn,h2_mi_match_yn,h2_all_match_yn,h3_la_match_yn,h3_mi_match_yn,h3_all_match_yn,Year,Month,Day,dayofweek,Hour,Minute,Second,dayofyear,season,Hour2,d1_match,d2_match,d3_match,h1_match,h2_match,h3_match
0,0,0,1,4,3,5,275,370,369,8,1,1,4,95,59,3,3,10,2,1,2,1608,275,1,4,139,618822,354805,1,275,274,274,216,369,368,297,216,369,368,297,...,True,True,True,True,True,True,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,2020,1,17,4,12,9,36,17,winter,afternoon,TTTTT,TFFFF,TFFFF,FFF,TFF,FFF
1,1,1,1,3,4,1,114,181,175,4,1,1,131,101,96,1,3,5,1,1,2,1608,275,1,4,133,571659,346213,0,114,109,56,1,175,152,56,1,175,152,56,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,False,True,False,False,2020,6,18,3,17,48,52,170,summer,afternoon,FFFFF,FFFFF,FFFFF,TTF,TFF,TFF
2,2,2,2,0,3,5,464,175,452,3,1,1,54,263,56,3,1,10,2,1,1,1600,94,1,4,53,399816,206408,0,464,463,450,377,175,152,56,1,452,451,450,...,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,2020,7,8,2,20,0,10,190,summer,night,FFFFF,TTFFF,FFFFF,TFF,FFF,TFF
3,3,3,2,0,2,5,703,705,704,3,1,1,72,227,2,1,3,5,1,1,2,1608,275,5,3,74,827967,572323,0,703,703,690,618,703,703,690,618,703,703,690,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,2020,1,13,0,18,9,34,13,winter,night,FFFFF,FFFFF,FFFFF,TFF,FFF,FFF
4,4,4,1,3,4,5,275,370,369,4,1,1,214,210,209,1,1,10,2,1,2,1608,275,1,4,74,831614,573899,0,275,274,274,216,369,368,297,216,369,368,297,...,True,True,True,True,True,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,2020,3,9,0,20,39,22,69,spring,night,TTTTT,TFFFF,TFFFF,FFF,FFF,FFF


In [ ]:
df_train = Rdata_train.copy()
df_test = Rdata_test.copy()
df_D = Rdata_D.copy()
df_H = Rdata_H.copy()
df_L = Rdata_L.copy()

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501951 entries, 0 to 501950
Data columns (total 97 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   Unnamed: 0                  501951 non-null  int64 
 1   id                          501951 non-null  int64 
 2   person_attribute_a          501951 non-null  int64 
 3   person_attribute_a_under    501951 non-null  int64 
 4   person_attribute_b          501951 non-null  int64 
 5   person_prefer_c             501951 non-null  int64 
 6   person_prefer_d_1           501951 non-null  int64 
 7   person_prefer_d_2           501951 non-null  int64 
 8   person_prefer_d_3           501951 non-null  int64 
 9   person_prefer_e             501951 non-null  int64 
 10  person_prefer_f             501951 non-null  int64 
 11  person_prefer_g             501951 non-null  int64 
 12  person_prefer_h_1           501951 non-null  int64 
 13  person_prefer_h_2           5

In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46404 entries, 0 to 46403
Data columns (total 96 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Unnamed: 0                  46404 non-null  int64 
 1   id                          46404 non-null  int64 
 2   person_attribute_a          46404 non-null  int64 
 3   person_attribute_a_under    46404 non-null  int64 
 4   person_attribute_b          46404 non-null  int64 
 5   person_prefer_c             46404 non-null  int64 
 6   person_prefer_d_1           46404 non-null  int64 
 7   person_prefer_d_2           46404 non-null  int64 
 8   person_prefer_d_3           46404 non-null  int64 
 9   person_prefer_e             46404 non-null  int64 
 10  person_prefer_f             46404 non-null  int64 
 11  person_prefer_g             46404 non-null  int64 
 12  person_prefer_h_1           46404 non-null  int64 
 13  person_prefer_h_2           46404 non-null  in

# 2. 범주형 encoding

In [ ]:
df_train.dtypes.value_counts()

int64     65
bool      24
object     8
dtype: int64

int형에 속하는 컬럼들에 특징 값 분포
*   특징 값 분포를 통해서 Label Encoding과 Ont-Hot Encoding 방식을 결정
*   가장 많은 특징 값을 갖는 것이 'person_rn' -> 300177


*   동일한 특징 값을 갖는 특징들은 ['person_prefer_f', 'person_prefer_g', 'Y'] 라는 것을 파악 ->  이 변수들은 볼 필요가 없다.
  *   모든 값에 대해서 동일한 값을 갖는 것을 주목해야 한다.
  *   이러한 특징 값들은 추후 모델 학습에서 Drop 시키고 학습을 실시 하겠습니다.

Ont-Hot Encoding
1. 순서가 없을 때 (ex. 국가명)
2. 고유값의 개수가 많지 않으면 효율적

Label Encoding
1. 순서의 의미가 있을때 (유치원, 초등학교, 대학교 같은 등급, 사원, 대리, 과장, 부장 같은 직급?? )
2. 고유값의 개수가 많을때 (One-hot Encoding은 당연히 메모리 소비가 많으므로 효율적이진 못하다. )
3. 선형 회귀와 같은 ML알고리즘에는 적용 불가
4. 트리 계열의 ML 알고리즘은 숫자의 이러한 특성을 반영하지 않으므로 가능
5. LabelEncoder는 dataframe의 열마다 접근해야 한다.

In [ ]:
df_train.select_dtypes('int64').apply(pd.Series.nunique, axis = 0)[df_train.select_dtypes('int64').apply(pd.Series.nunique, axis = 0) == 1]

person_prefer_f    1
person_prefer_g    1
Year               1
dtype: int64

In [ ]:
df_train.columns

Index(['Unnamed: 0', 'id', 'person_attribute_a', 'person_attribute_a_under',
       'person_attribute_b', 'person_prefer_c', 'person_prefer_d_1',
       'person_prefer_d_2', 'person_prefer_d_3', 'person_prefer_e',
       'person_prefer_f', 'person_prefer_g', 'person_prefer_h_1',
       'person_prefer_h_2', 'person_prefer_h_3', 'contents_attribute_i',
       'contents_attribute_a', 'contents_attribute_j_under',
       'contents_attribute_j', 'contents_attribute_c', 'contents_attribute_k',
       'contents_attribute_l', 'contents_attribute_d', 'contents_attribute_m',
       'contents_attribute_e', 'contents_attribute_h', 'person_rn',
       'contents_rn', 'target', 'person_prefer_d_1_세', 'person_prefer_d_1_소',
       'person_prefer_d_1_중', 'person_prefer_d_1_대', 'person_prefer_d_2_세',
       'person_prefer_d_2_소', 'person_prefer_d_2_중', 'person_prefer_d_2_대',
       'person_prefer_d_3_세', 'person_prefer_d_3_소', 'person_prefer_d_3_중',
       'person_prefer_d_3_대', 'contents_attribute_d_세'

In [ ]:
for column in df_train.select_dtypes(include = 'bool').columns :
    df_train[column] = df_train[column].astype(int)
    
for column in df_test.select_dtypes(include = 'bool').columns :
    df_test[column] = df_test[column].astype(int)

In [ ]:
df_train2 = df_train.drop(['person_prefer_f', 'person_prefer_g', 'Year', 'Unnamed: 0', 'id', 'Minute', 'Second'], axis = 1)
df_test2 = df_test.drop(['person_prefer_f', 'person_prefer_g', 'Year', 'Unnamed: 0', 'id', 'Minute', 'Second'], axis = 1)
print(df_train2.shape)
print(df_test2.shape)

(501951, 90)
(46404, 89)


In [ ]:
df_train2.dtypes.value_counts()

int64     82
object     8
dtype: int64

In [ ]:
df_train2.select_dtypes(include = 'object').columns

Index(['season', 'Hour2', 'd1_match', 'd2_match', 'd3_match', 'h1_match',
       'h2_match', 'h3_match'],
      dtype='object')

In [ ]:
df_train2.columns

Index(['person_attribute_a', 'person_attribute_a_under', 'person_attribute_b',
       'person_prefer_c', 'person_prefer_d_1', 'person_prefer_d_2',
       'person_prefer_d_3', 'person_prefer_e', 'person_prefer_h_1',
       'person_prefer_h_2', 'person_prefer_h_3', 'contents_attribute_i',
       'contents_attribute_a', 'contents_attribute_j_under',
       'contents_attribute_j', 'contents_attribute_c', 'contents_attribute_k',
       'contents_attribute_l', 'contents_attribute_d', 'contents_attribute_m',
       'contents_attribute_e', 'contents_attribute_h', 'person_rn',
       'contents_rn', 'target', 'person_prefer_d_1_세', 'person_prefer_d_1_소',
       'person_prefer_d_1_중', 'person_prefer_d_1_대', 'person_prefer_d_2_세',
       'person_prefer_d_2_소', 'person_prefer_d_2_중', 'person_prefer_d_2_대',
       'person_prefer_d_3_세', 'person_prefer_d_3_소', 'person_prefer_d_3_중',
       'person_prefer_d_3_대', 'contents_attribute_d_세',
       'contents_attribute_d_소', 'contents_attribute_d_중',
    

In [ ]:
numerical_feats = ['Month', 'Day', 'dayofweek', 'Hour', 'dayofyear']

ordinal_feats = ['person_attribute_a_under', 'person_attribute_b', 'person_prefer_e', 'contents_attribute_e']

categorical_feats_oe =  ['person_prefer_d_1', 'person_prefer_d_2', 'person_prefer_d_3',
                        'person_prefer_h_1', 'person_prefer_h_2', 'person_prefer_h_3', 'contents_attribute_l',
                        'contents_attribute_d', 'contents_attribute_h', 'person_rn', 'contents_rn', 
                        'person_prefer_d_1_세', 'person_prefer_d_1_소', 'person_prefer_d_1_중',
                        'person_prefer_d_1_대', 'person_prefer_d_2_세', 'person_prefer_d_2_소',
                        'person_prefer_d_2_중', 'person_prefer_d_2_대', 'person_prefer_d_3_세',
                        'person_prefer_d_3_소', 'person_prefer_d_3_중', 'person_prefer_d_3_대',
                        'contents_attribute_d_세', 'contents_attribute_d_소',
                        'contents_attribute_d_중', 'contents_attribute_d_대',
                        'person_prefer_h_1_중', 'person_prefer_h_1_대', 'person_prefer_h_2_중',
                        'person_prefer_h_2_대', 'person_prefer_h_3_중', 'person_prefer_h_3_대',
                        'contents_attribute_h_중', 'contents_attribute_h_대',
                        'contents_attribute_l_세', 'contents_attribute_l_소',
                        'contents_attribute_l_중', 'contents_attribute_l_대']

print("Number of numerical_feats : ", len(numerical_feats))
print("Number of ordinal_feats : ", len(ordinal_feats))
print("Number of categorical_feats_oe : ", len(categorical_feats_oe))

Number of numerical_feats :  5
Number of ordinal_feats :  4
Number of categorical_feats_oe :  39


In [ ]:
temp = numerical_feats + ordinal_feats + categorical_feats_oe
categorical_feats_gd_ohe = df_train2.columns[~df_train2.columns.isin(temp)]
categorical_feats_gd_ohe = pd.DataFrame(categorical_feats_gd_ohe)
categorical_feats_gd_ohe = categorical_feats_gd_ohe.loc[categorical_feats_gd_ohe[0] != 'target', :]
categorical_feats_gd_ohe = categorical_feats_gd_ohe[0]
categorical_feats_gd_ohe = categorical_feats_gd_ohe.values.tolist()
print("Number of categorical_feats_gd_ohe : ", len(categorical_feats_gd_ohe))

categorical_feats_gd = ['season', 'Hour2', 'd1_match', 'd2_match', 'd3_match', 'h1_match', 'h2_match', 'h3_match']
categorical_feats_ohe = categorical_feats_gd_ohe.copy()
for obj in categorical_feats_gd :
  categorical_feats_ohe.remove(obj)

print("Number of categorical_feats_gd : ", len(categorical_feats_gd))
print("Number of categorical_feats_ohe : ", len(categorical_feats_ohe))

Number of categorical_feats_gd_ohe :  41
Number of categorical_feats_gd :  8
Number of categorical_feats_ohe :  33


In [ ]:
categorical_feats_gd

['season',
 'Hour2',
 'd1_match',
 'd2_match',
 'd3_match',
 'h1_match',
 'h2_match',
 'h3_match']

In [ ]:
categorical_feats_ohe

['person_attribute_a',
 'person_prefer_c',
 'contents_attribute_i',
 'contents_attribute_a',
 'contents_attribute_j_under',
 'contents_attribute_j',
 'contents_attribute_c',
 'contents_attribute_k',
 'contents_attribute_m',
 'd1_la_match_yn',
 'd1_mi_match_yn',
 'd1_sm_match_yn',
 'd1_sub_match_yn',
 'd1_all_match_yn',
 'd2_la_match_yn',
 'd2_mi_match_yn',
 'd2_sm_match_yn',
 'd2_sub_match_yn',
 'd2_all_match_yn',
 'd3_la_match_yn',
 'd3_mi_match_yn',
 'd3_sm_match_yn',
 'd3_sub_match_yn',
 'd3_all_match_yn',
 'h1_la_match_yn',
 'h1_mi_match_yn',
 'h1_all_match_yn',
 'h2_la_match_yn',
 'h2_mi_match_yn',
 'h2_all_match_yn',
 'h3_la_match_yn',
 'h3_mi_match_yn',
 'h3_all_match_yn']

In [ ]:
categorical_feats_oe

['person_prefer_d_1',
 'person_prefer_d_2',
 'person_prefer_d_3',
 'person_prefer_h_1',
 'person_prefer_h_2',
 'person_prefer_h_3',
 'contents_attribute_l',
 'contents_attribute_d',
 'contents_attribute_h',
 'person_rn',
 'contents_rn',
 'person_prefer_d_1_세',
 'person_prefer_d_1_소',
 'person_prefer_d_1_중',
 'person_prefer_d_1_대',
 'person_prefer_d_2_세',
 'person_prefer_d_2_소',
 'person_prefer_d_2_중',
 'person_prefer_d_2_대',
 'person_prefer_d_3_세',
 'person_prefer_d_3_소',
 'person_prefer_d_3_중',
 'person_prefer_d_3_대',
 'contents_attribute_d_세',
 'contents_attribute_d_소',
 'contents_attribute_d_중',
 'contents_attribute_d_대',
 'person_prefer_h_1_중',
 'person_prefer_h_1_대',
 'person_prefer_h_2_중',
 'person_prefer_h_2_대',
 'person_prefer_h_3_중',
 'person_prefer_h_3_대',
 'contents_attribute_h_중',
 'contents_attribute_h_대',
 'contents_attribute_l_세',
 'contents_attribute_l_소',
 'contents_attribute_l_중',
 'contents_attribute_l_대']

In [ ]:
numerical_feats

['Month', 'Day', 'dayofweek', 'Hour', 'dayofyear']

In [ ]:
# OrdinalEncoder

oe = OrdinalEncoder(categorical_feats_gd)
df_train2[categorical_feats_gd] = oe.fit_transform(df_train2[categorical_feats_gd], df_train2['target'])
df_test2[categorical_feats_gd] = oe.transform(df_test2[categorical_feats_gd])

df_train2[categorical_feats_gd] = df_train2[categorical_feats_gd].astype('int64')
df_test2[categorical_feats_gd] = df_test2[categorical_feats_gd].astype('int64')

In [ ]:
df_train2.head()

,person_attribute_a,person_attribute_a_under,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_under,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,target,person_prefer_d_1_세,person_prefer_d_1_소,person_prefer_d_1_중,person_prefer_d_1_대,person_prefer_d_2_세,person_prefer_d_2_소,person_prefer_d_2_중,person_prefer_d_2_대,person_prefer_d_3_세,person_prefer_d_3_소,person_prefer_d_3_중,person_prefer_d_3_대,contents_attribute_d_세,contents_attribute_d_소,contents_attribute_d_중,...,contents_attribute_l_소,contents_attribute_l_중,contents_attribute_l_대,d1_la_match_yn,d1_mi_match_yn,d1_sm_match_yn,d1_sub_match_yn,d1_all_match_yn,d2_la_match_yn,d2_mi_match_yn,d2_sm_match_yn,d2_sub_match_yn,d2_all_match_yn,d3_la_match_yn,d3_mi_match_yn,d3_sm_match_yn,d3_sub_match_yn,d3_all_match_yn,h1_la_match_yn,h1_mi_match_yn,h1_all_match_yn,h2_la_match_yn,h2_mi_match_yn,h2_all_match_yn,h3_la_match_yn,h3_mi_match_yn,h3_all_match_yn,Month,Day,dayofweek,Hour,dayofyear,season,Hour2,d1_match,d2_match,d3_match,h1_match,h2_match,h3_match
0,1,4,3,5,275,370,369,8,4,95,59,3,3,10,2,1,2,1608,275,1,4,139,618822,354805,1,275,274,274,216,369,368,297,216,369,368,297,216,275,274,274,...,1606,1605,2016,1,1,1,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,17,4,12,17,1,1,1,1,1,1,1,1
1,1,3,4,1,114,181,175,4,131,101,96,1,3,5,1,1,2,1608,275,1,4,133,571659,346213,0,114,109,56,1,175,152,56,1,175,152,56,1,275,274,274,...,1606,1605,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,6,18,3,17,170,2,1,2,2,2,2,1,2
2,2,0,3,5,464,175,452,3,54,263,56,3,1,10,2,1,1,1600,94,1,4,53,399816,206408,0,464,463,450,377,175,152,56,1,452,451,450,377,92,91,56,...,1595,1572,2016,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,7,8,2,20,190,2,2,2,3,2,3,2,2
3,2,0,2,5,703,705,704,3,72,227,2,1,3,5,1,1,2,1608,275,5,3,74,827967,572323,0,703,703,690,618,703,703,690,618,703,703,690,618,275,274,274,...,1606,1605,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,13,0,18,13,1,2,2,2,2,3,2,1
4,1,3,4,5,275,370,369,4,214,210,209,1,1,10,2,1,2,1608,275,1,4,74,831614,573899,0,275,274,274,216,369,368,297,216,369,368,297,216,275,274,274,...,1606,1605,2016,1,1,1,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3,9,0,20,69,3,2,1,1,1,1,2,1


In [ ]:
print(df_train2.dtypes.value_counts())
print(df_test2.dtypes.value_counts())

int64    90
dtype: int64
int64    89
dtype: int64


In [ ]:
###################################################################################################################
# 여기서부터

# a = pd.get_dummies(df_train2[categorical_feats_gd])
# df_train3 = pd.concat([df_train2, a], axis = 1)
# for obj in categorical_feats_gd :
#     df_train3 = df_train3.drop(obj, axis = 1)
    
# print(df_train3.shape)

# b = pd.get_dummies(df_test2[categorical_feats_gd])
# df_test3 = pd.concat([df_test2, b], axis = 1)
# for obj in categorical_feats_gd :
#     df_test3 = df_test3.drop(obj, axis = 1)

# df_test3['season_fall'] = 0
# df_test3['season_spring'] = 0
# df_test3['season_summer'] = 0

# print(df_test3.shape)
###################################################################################################################

In [ ]:
# df_train3.dtypes.value_counts()

In [ ]:
###################################################################################################################
# One Hot Enconding

# ohe = OneHotEncoder()

# c = ohe.fit_transform(df_train3[categorical_feats_ohe]).toarray()
# d = ohe.transform(df_test3[categorical_feats_ohe]).toarray()

# ohe_columns = ohe.get_feature_names(categorical_feats_ohe)
# cc = pd.DataFrame(c, columns = ohe_columns)
# dd = pd.DataFrame(d, columns = ohe_columns)
# print(cc.shape)
# print(dd.shape)


# df_train4 = pd.concat([df_train3, cc], axis = 1)
# for obj in categorical_feats_ohe :
#     df_train4 = df_train4.drop(obj, axis = 1)

# print(len(categorical_feats_ohe))
# print(df_train3.shape)
# print(df_train4.shape)

# df_test4 = pd.concat([df_test3, dd], axis = 1)
# for obj in categorical_feats_ohe :
 #    df_test4 = df_test4.drop(obj, axis = 1)

# print(len(categorical_feats_ohe))
# print(df_test3.shape)
# print(df_test4.shape)
###################################################################################################################

In [ ]:
###################################################################################################################
# 숫자 -> 문자

# df_train4[categorical_feats_oe] = df_train4[categorical_feats_oe].astype(str)
# print(df_train4.dtypes)

# df_test4[categorical_feats_oe] = df_test4[categorical_feats_oe].astype(str)
# print(df_test4.dtypes)
###################################################################################################################

In [ ]:
###################################################################################################################
# OrdinalEncoder

# oe = OrdinalEncoder(categorical_feats_oe)
# df_train4[categorical_feats_oe] = oe.fit_transform(df_train4[categorical_feats_oe], df_train4['target'])
# df_test4[categorical_feats_oe] = oe.transform(df_test4[categorical_feats_oe])

# df_train4['person_rn'] = df_train4['person_rn'].astype('int64')
# df_train4['contents_rn'] = df_train4['contents_rn'].astype('int64')
# df_test4['person_rn'] = df_test4['person_rn'].astype('int64')
# df_test4['contents_rn'] = df_test4['contents_rn'].astype('int64')
###################################################################################################################

In [ ]:
df_train3 = df_train2.copy()
df_test3 = df_test2.copy()

# 3. 클러스터링 및 데이터 분할

In [ ]:
# 클러스터링 구성
# 타겟을 결정짓는 뚜렷한 특징을 갖는 피쳐를 찾지 못해 clustering 시도

kmeans_train = df_train3.drop(['target'], axis = 1)
kmeans = KMeans(n_clusters = 15, init = 'k-means++', n_init = 10, max_iter = 300, tol = 1e-04, random_state = 39).fit(kmeans_train)
df_train3['cluster'] = kmeans.predict(kmeans_train)
df_test3['cluster'] = kmeans.predict(df_test3)

In [ ]:
df_train3['cluster'] = df_train3['cluster'].astype('int64')
df_test3['cluster'] = df_test3['cluster'].astype('int64')

df_train3.dtypes.value_counts()

int64    91
dtype: int64

In [ ]:
df_train4 = df_train3.copy()
df_test4 = df_test3.copy()

In [ ]:
# StandardScale
# 컬럼 정규화

temp_columns2 = numerical_feats
scaler = StandardScaler()
df_train4[temp_columns2] = scaler.fit_transform(df_train4[temp_columns2])
df_test4[temp_columns2] = scaler.transform(df_test4[temp_columns2])

In [ ]:
print(df_train4.dtypes.value_counts())
print(df_test4.dtypes.value_counts())

int64      86
float64     5
dtype: int64
int64      85
float64     5
dtype: int64


In [ ]:
numerical_feats

['Month', 'Day', 'dayofweek', 'Hour', 'dayofyear']

In [ ]:
sorted(df_train4['Month'].unique())

[-1.553057947212434,
 -1.238148108969224,
 -0.9232382707260137,
 -0.6083284324828035,
 -0.2934185942395934,
 0.021491244003616763,
 0.3364010822468269,
 0.651310920490037,
 0.9662207587332472,
 1.2811305969764575,
 1.5960404352196675]

In [ ]:
df_train6_1 = df_train4.loc[df_train4['Month'] < 0.7, :]
df_train6_1.reset_index(drop = True, inplace = True)
X_part_train = df_train6_1.drop('target', axis = 1)
y_part_train = df_train6_1['target']

df_train6_2 = df_train4.loc[(df_train4['Month'] > 0.7) & (df_train4['Month'] < 1.4)]
df_train6_2.reset_index(drop = True, inplace = True)
X_val = df_train6_2.drop('target', axis = 1)
y_val = df_train6_2['target']

df_train6_3 = df_train4.loc[df_train4['Month']> 1.4]
df_train6_3.reset_index(drop = True, inplace = True)
X_part_test = df_train6_3.drop('target', axis = 1)
y_part_test = df_train6_3['target']

df_train6_4 = df_train4.loc[df_train4['Month'] < 1.4]
df_train6_4.reset_index(drop = True, inplace = True)
X_train = df_train6_4.drop('target', axis = 1)
y_train = df_train6_4['target']

X_test = df_test4.copy()

print(X_part_train.shape)
print(y_part_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_part_test.shape)
print(y_part_test.shape)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(368974, 90)
(368974,)
(87998, 90)
(87998,)
(44979, 90)
(44979,)
(456972, 90)
(456972,)
(46404, 90)


In [ ]:
tt1 = X_train.columns.tolist() 
tt2 = numerical_feats
print(len(tt1))
print(len(tt2))
tt3 = []
for i in tt1 :
  if i not in tt2 :
    tt3.append(i)
print(len(tt3))
print(tt3)

cat_cols = tt3

90
5
85
['person_attribute_a', 'person_attribute_a_under', 'person_attribute_b', 'person_prefer_c', 'person_prefer_d_1', 'person_prefer_d_2', 'person_prefer_d_3', 'person_prefer_e', 'person_prefer_h_1', 'person_prefer_h_2', 'person_prefer_h_3', 'contents_attribute_i', 'contents_attribute_a', 'contents_attribute_j_under', 'contents_attribute_j', 'contents_attribute_c', 'contents_attribute_k', 'contents_attribute_l', 'contents_attribute_d', 'contents_attribute_m', 'contents_attribute_e', 'contents_attribute_h', 'person_rn', 'contents_rn', 'person_prefer_d_1_세', 'person_prefer_d_1_소', 'person_prefer_d_1_중', 'person_prefer_d_1_대', 'person_prefer_d_2_세', 'person_prefer_d_2_소', 'person_prefer_d_2_중', 'person_prefer_d_2_대', 'person_prefer_d_3_세', 'person_prefer_d_3_소', 'person_prefer_d_3_중', 'person_prefer_d_3_대', 'contents_attribute_d_세', 'contents_attribute_d_소', 'contents_attribute_d_중', 'contents_attribute_d_대', 'person_prefer_h_1_중', 'person_prefer_h_1_대', 'person_prefer_h_2_중', 'person_

In [ ]:
temp = X_part_train.columns[X_part_train.nunique() > 2].tolist()
temp2 = set(temp)
cat_cols2 = set(cat_cols)
cat_cols3 = list(temp2 & cat_cols2)
print(len(cat_cols3))

58


# 4. 1차 모델 생성 (randomforest, xgboost, lightgbm, extratree, catboost, tabnet, hgbm)

### 1) randomforest

In [ ]:
def objective(trial : Trial) -> float :

    params_cat = {
        "random_state" : 39,
        'min_samples_split' : trial.suggest_int('min_samples_split', 1, 150),
        'min_samples_leaf' : trial.suggest_int('min_samples_leaf', 1, 60),
        "n_estimators":trial.suggest_int("n_estimators", 100, 1000),
        "max_depth":trial.suggest_int("max_depth", 10, 16)
  }
    
    model = RandomForestClassifier(**params_cat)
    model.fit(X_part_train, y_part_train)

    rf_pred = model.predict(X_val)
    f1 = f1_score(y_val, rf_pred)
    
    return f1

In [ ]:
sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "rf_parameter_opt",
    direction = "maximize",
    sampler = sampler)
study.optimize(objective, n_trials = 20)

[I 2022-01-16 12:54:29,534] A new study created in memory with name: rf_parameter_opt
[I 2022-01-16 12:56:55,269] Trial 0 finished with value: 0.5919312315838448 and parameters: {'min_samples_split': 83, 'min_samples_leaf': 48, 'n_estimators': 839, 'max_depth': 2}. Best is trial 0 with value: 0.5919312315838448.
[I 2022-01-16 13:01:50,215] Trial 1 finished with value: 0.6161261387829207 and parameters: {'min_samples_split': 91, 'min_samples_leaf': 32, 'n_estimators': 517, 'max_depth': 8}. Best is trial 1 with value: 0.6161261387829207.
[I 2022-01-16 13:15:22,256] Trial 2 finished with value: 0.6359537927888747 and parameters: {'min_samples_split': 95, 'min_samples_leaf': 56, 'n_estimators': 834, 'max_depth': 16}. Best is trial 2 with value: 0.6359537927888747.
[I 2022-01-16 13:29:18,499] Trial 3 finished with value: 0.6350980850566407 and parameters: {'min_samples_split': 138, 'min_samples_leaf': 25, 'n_estimators': 844, 'max_depth': 16}. Best is trial 2 with value: 0.6359537927888747.

In [ ]:
print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)

Best Score : 0.6359537927888747
Best trial : {'min_samples_split': 95, 'min_samples_leaf': 56, 'n_estimators': 834, 'max_depth': 16}


In [ ]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [ ]:
# 파라미터들과의 관계
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

In [ ]:
n_fold = 5
cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 39)

rf_val = np.zeros((X_train.shape[0], 2))
rf_part_test = np.zeros((X_part_test.shape[0], 2))
rf_test = np.zeros((X_test.shape[0], 2))

print(rf_val.shape)
print(rf_part_test.shape)
print(rf_test.shape)

for i, (i_trn, i_val) in enumerate(cv.split(X_train, y_train), 1):
    print(f'training model for CV #{i}')
    optuna_rf = RandomForestClassifier(
                        random_state = 39,
                        min_samples_split = 95,
                        min_samples_leaf = 56,
                        n_estimators = 834,
                        max_depth = 16)

    optuna_rf.fit(X_train.loc[i_trn, :], y_train[i_trn])

    rf_val[i_val, :] = optuna_rf.predict_proba(X_train.loc[i_val, :])
    rf_part_test += optuna_rf.predict_proba(X_part_test) / n_fold
    rf_test += optuna_rf.predict_proba(X_test) / n_fold

(456972, 2)
(44979, 2)
(46404, 2)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5


In [ ]:
print(rf_val.shape)
print(rf_part_test.shape)
print(rf_test.shape)

rf_pred = rf_val[:, 1]
rf_pred = np.where(rf_pred >= 0.5, 1, 0)
print(f1_score(y_train, rf_pred))

rf_pred2 = rf_part_test[:, 1]
rf_pred2 = np.where(rf_pred2 >= 0.5, 1, 0)
print(f1_score(y_part_test, rf_pred2))

(456972, 2)
(44979, 2)
(46404, 2)
0.63256538953144
0.6212494378172317


In [ ]:
# 저장

rf_val_file = pd.DataFrame(rf_val, columns = ['0', '1'])
rf_part_test_file = pd.DataFrame(rf_part_test, columns = ['0', '1'])
rf_test_file = pd.DataFrame(rf_test, columns = ['0', '1'])
rf_val_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/rf_val.csv', index = False)
rf_part_test_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/rf_part_test.csv', index = False)
rf_test_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/rf_test.csv', index = False)

### 2) lightgbm

In [ ]:
n_fold = 5
cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 39)

lgbm_val = np.zeros((X_train.shape[0], 2))
lgbm_part_test = np.zeros((X_part_test.shape[0], 2))
lgbm_test = np.zeros((X_test.shape[0], 2))

print(lgbm_val.shape)
print(lgbm_part_test.shape)
print(lgbm_test.shape)

for i, (i_trn, i_val) in enumerate(cv.split(X_train, y_train), 1):
    print(f'training model for CV #{i}')
    optuna_lgbm = LGBMClassifier(
                        random_state = 39,
                        n_estimators = 1447, 
                        reg_alpha = 1.9789105242232932, 
                        reg_lambda = 0.0026602983980182904, 
                        max_depth = 14, 
                        num_leaves = 87, 
                        colsample_bytree = 0.992812938438909, 
                        subsample = 0.8599496113145693, 
                        subsample_freq = 10, 
                        min_child_samples = 41, 
                        max_bin = 335,
                        verbose = 100)

    optuna_lgbm.fit(X_train.loc[i_trn, :], y_train[i_trn], eval_set = [(X_train.loc[i_val, :], y_train[i_val])], early_stopping_rounds = 100, eval_metric = 'auc')

    lgbm_val[i_val, :] = optuna_lgbm.predict_proba(X_train.loc[i_val, :])
    lgbm_part_test += optuna_lgbm.predict_proba(X_part_test) / n_fold
    lgbm_test += optuna_lgbm.predict_proba(X_test) / n_fold

(456972, 2)
(44979, 2)
(46404, 2)
training model for CV #1
[1]	valid_0's auc: 0.637257	valid_0's binary_logloss: 0.687253
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.640047	valid_0's binary_logloss: 0.682472
[3]	valid_0's auc: 0.64109	valid_0's binary_logloss: 0.678572
[4]	valid_0's auc: 0.643567	valid_0's binary_logloss: 0.674988
[5]	valid_0's auc: 0.64466	valid_0's binary_logloss: 0.672186
[6]	valid_0's auc: 0.645299	valid_0's binary_logloss: 0.669771
[7]	valid_0's auc: 0.646512	valid_0's binary_logloss: 0.66759
[8]	valid_0's auc: 0.647023	valid_0's binary_logloss: 0.665845
[9]	valid_0's auc: 0.647944	valid_0's binary_logloss: 0.664255
[10]	valid_0's auc: 0.648668	valid_0's binary_logloss: 0.662789
[11]	valid_0's auc: 0.64975	valid_0's binary_logloss: 0.661537
[12]	valid_0's auc: 0.650312	valid_0's binary_logloss: 0.660439
[13]	valid_0's auc: 0.650858	valid_0's binary_logloss: 0.659514
[14]	valid_0's auc: 0.651496	valid_0's binary_logloss: 0.65

In [ ]:
print(lgbm_val.shape)
print(lgbm_part_test.shape)
print(lgbm_test.shape)

lgbm_pred = lgbm_val[:, 1]
lgbm_pred = np.where(lgbm_pred >= 0.5, 1, 0)
print(f1_score(y_train, lgbm_pred))

lgbm_pred2 = lgbm_part_test[:, 1]
lgbm_pred2 = np.where(lgbm_pred2 >= 0.5, 1, 0)
print(f1_score(y_part_test, lgbm_pred2))

(456972, 2)
(44979, 2)
(46404, 2)
0.6490524834906404
0.6149492183042338


In [ ]:
# 저장

lgbm_val_file = pd.DataFrame(lgbm_val, columns = ['0', '1'])
lgbm_part_test_file = pd.DataFrame(lgbm_part_test, columns = ['0', '1'])
lgbm_test_file = pd.DataFrame(lgbm_test, columns = ['0', '1'])
lgbm_val_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/lgbm_val.csv', index = False)
lgbm_part_test_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/lgbm_part_test.csv', index = False)
lgbm_test_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/lgbm_test.csv', index = False)

### 3) xgboost

In [ ]:
def objective(trial : Trial) -> float :

    params_xgb = {
        "random_state" : 39,
        'max_depth' : trial.suggest_int("max_depth", 1, 16),
        'colsample_bytree' : trial.suggest_float("colsample_bytree", 0.1, 1.0),
        'colsample_bylevel' : trial.suggest_float("colsample_bylevel", 0.1, 1.0),
        'colsample_bynode' : trial.suggest_float("colsample_bynode", 0.1, 1.0),
        'min_child_weight' : trial.suggest_float("min_child_weight", 0.1, 300.0),
        'gamma' : trial.suggest_loguniform('gamma', 1e-3, 10.0),
        'subsample' : trial.suggest_float("subsample", 0.1, 1.0),
       'reg_alpha' : trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda' : trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        "n_estimators" : trial.suggest_int("n_estimators", 100, 1000),
        "max_depth" : trial.suggest_int("max_depth", 1, 16)
  }
    
    model = xgb.XGBClassifier(**params_cat)
    model.fit(X_part_train, y_part_train, eval_set = [(X_val, y_val)], early_stopping_rounds = 100, eval_metric = 'auc')

    xgb_pred = model.predict(X_val)
    f1 = f1_score(y_val, xgb_pred)
    
    return f1

In [ ]:
sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "xgb_parameter_opt",
    direction = "maximize",
    sampler = sampler)
study.optimize(objective, n_trials = 20)

[I 2022-01-17 11:49:56,711] A new study created in memory with name: xgb_parameter_opt


[0]	validation_0-auc:0.618217
Will train until validation_0-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.626592
[2]	validation_0-auc:0.631875
[3]	validation_0-auc:0.633715
[4]	validation_0-auc:0.635715
[5]	validation_0-auc:0.637304
[6]	validation_0-auc:0.639122
[7]	validation_0-auc:0.640122
[8]	validation_0-auc:0.640592
[9]	validation_0-auc:0.64107
[10]	validation_0-auc:0.64191
[11]	validation_0-auc:0.642037
[12]	validation_0-auc:0.643385
[13]	validation_0-auc:0.643996
[14]	validation_0-auc:0.644416
[15]	validation_0-auc:0.645012
[16]	validation_0-auc:0.645402
[17]	validation_0-auc:0.645724
[18]	validation_0-auc:0.646024
[19]	validation_0-auc:0.646298
[20]	validation_0-auc:0.647002
[21]	validation_0-auc:0.647594
[22]	validation_0-auc:0.647889
[23]	validation_0-auc:0.648427
[24]	validation_0-auc:0.648864
[25]	validation_0-auc:0.649137
[26]	validation_0-auc:0.649457
[27]	validation_0-auc:0.649686
[28]	validation_0-auc:0.650331
[29]	validation_0-auc:0.65082
[30]	validation_0-

[I 2022-01-17 11:56:31,484] Trial 0 finished with value: 0.6302902491814196 and parameters: {'max_depth': 9, 'colsample_bytree': 0.8181091191543447, 'colsample_bylevel': 0.8383616919085431, 'colsample_bynode': 0.20984488014718103, 'min_child_weight': 180.64040183675223, 'gamma': 0.12649062354229204, 'subsample': 0.5175175652290616, 'reg_alpha': 0.07687461923467091, 'reg_lambda': 0.33950906260567937, 'n_estimators': 934}. Best is trial 0 with value: 0.6302902491814196.


[0]	validation_0-auc:0.626649
Will train until validation_0-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.633403
[2]	validation_0-auc:0.636792
[3]	validation_0-auc:0.640461
[4]	validation_0-auc:0.642111
[5]	validation_0-auc:0.642859
[6]	validation_0-auc:0.642939
[7]	validation_0-auc:0.644539
[8]	validation_0-auc:0.645205
[9]	validation_0-auc:0.645954
[10]	validation_0-auc:0.646655
[11]	validation_0-auc:0.647353
[12]	validation_0-auc:0.64804
[13]	validation_0-auc:0.648825
[14]	validation_0-auc:0.649196
[15]	validation_0-auc:0.64945
[16]	validation_0-auc:0.649975
[17]	validation_0-auc:0.65046
[18]	validation_0-auc:0.65098
[19]	validation_0-auc:0.651507
[20]	validation_0-auc:0.651993
[21]	validation_0-auc:0.652536
[22]	validation_0-auc:0.652918
[23]	validation_0-auc:0.653382
[24]	validation_0-auc:0.653619
[25]	validation_0-auc:0.654289
[26]	validation_0-auc:0.654548
[27]	validation_0-auc:0.654972
[28]	validation_0-auc:0.65543
[29]	validation_0-auc:0.65573
[30]	validation_0-auc

[I 2022-01-17 12:14:26,948] Trial 1 finished with value: 0.6447430356455133 and parameters: {'max_depth': 14, 'colsample_bytree': 0.9499975771450039, 'colsample_bylevel': 0.9276266278535859, 'colsample_bynode': 0.4733477167159189, 'min_child_weight': 247.76100749471476, 'gamma': 6.095937074798688, 'subsample': 0.6652414479118475, 'reg_alpha': 0.010211256122219953, 'reg_lambda': 0.23410118458859783, 'n_estimators': 875}. Best is trial 1 with value: 0.6447430356455133.


...

[I 2022-01-17 13:43:15,228] Trial 17 finished with value: 0.6413744045181178 and parameters: {'max_depth': 10, 'colsample_bytree': 0.6858542465483815, 'colsample_bylevel': 0.9318546533113978, 'colsample_bynode': 0.6838869119011988, 'min_child_weight': 86.47302416230326, 'gamma': 0.7320162179573568, 'subsample': 0.9007404650460684, 'reg_alpha': 0.003894494448163446, 'reg_lambda': 0.02081542685419675, 'n_estimators': 748}. Best is trial 1 with value: 0.6447430356455133.


[0]	validation_0-auc:0.617085
Will train until validation_0-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.628821
[2]	validation_0-auc:0.630401
[3]	validation_0-auc:0.636571
[4]	validation_0-auc:0.638601
[5]	validation_0-auc:0.638374
[6]	validation_0-auc:0.640206
[7]	validation_0-auc:0.641473
[8]	validation_0-auc:0.64226
[9]	validation_0-auc:0.643161
[10]	validation_0-auc:0.64346
[11]	validation_0-auc:0.643733
[12]	validation_0-auc:0.644453
[13]	validation_0-auc:0.645134
[14]	validation_0-auc:0.646015
[15]	validation_0-auc:0.646592
[16]	validation_0-auc:0.64724
[17]	validation_0-auc:0.647816
[18]	validation_0-auc:0.647856
[19]	validation_0-auc:0.648249
[20]	validation_0-auc:0.648843
[21]	validation_0-auc:0.649946
[22]	validation_0-auc:0.650188
[23]	validation_0-auc:0.650779
[24]	validation_0-auc:0.651476
[25]	validation_0-auc:0.651769
[26]	validation_0-auc:0.652271
[27]	validation_0-auc:0.652803
[28]	validation_0-auc:0.653247
[29]	validation_0-auc:0.653586
[30]	validation_0-

[I 2022-01-17 13:47:57,596] Trial 18 finished with value: 0.6401704107028355 and parameters: {'max_depth': 13, 'colsample_bytree': 0.35691308803862754, 'colsample_bylevel': 0.47842473186754925, 'colsample_bynode': 0.5551062966777428, 'min_child_weight': 245.55593069866597, 'gamma': 0.0035400663375185983, 'subsample': 0.8186365874710574, 'reg_alpha': 0.023394383400966674, 'reg_lambda': 2.0380380389942045, 'n_estimators': 344}. Best is trial 1 with value: 0.6447430356455133.


[0]	validation_0-auc:0.624541
Will train until validation_0-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.630502
[2]	validation_0-auc:0.632914
[3]	validation_0-auc:0.634139
[4]	validation_0-auc:0.635092
[5]	validation_0-auc:0.635575
[6]	validation_0-auc:0.637382
[7]	validation_0-auc:0.638374
[8]	validation_0-auc:0.638832
[9]	validation_0-auc:0.6395
[10]	validation_0-auc:0.640301
[11]	validation_0-auc:0.640911
[12]	validation_0-auc:0.641276
[13]	validation_0-auc:0.641883
[14]	validation_0-auc:0.642388
[15]	validation_0-auc:0.642763
[16]	validation_0-auc:0.643416
[17]	validation_0-auc:0.644051
[18]	validation_0-auc:0.644433
[19]	validation_0-auc:0.644954
[20]	validation_0-auc:0.645473
[21]	validation_0-auc:0.646214
[22]	validation_0-auc:0.646845
[23]	validation_0-auc:0.647084
[24]	validation_0-auc:0.647339
[25]	validation_0-auc:0.647863
[26]	validation_0-auc:0.648183
[27]	validation_0-auc:0.649062
[28]	validation_0-auc:0.649322
[29]	validation_0-auc:0.650022
[30]	validation_0

[I 2022-01-17 13:55:09,875] Trial 19 finished with value: 0.6309094716885465 and parameters: {'max_depth': 7, 'colsample_bytree': 0.995450394399395, 'colsample_bylevel': 0.6420190680971003, 'colsample_bynode': 0.6954896059585103, 'min_child_weight': 48.66407693102346, 'gamma': 4.400136492272196, 'subsample': 0.6403915498105204, 'reg_alpha': 0.22546368454643093, 'reg_lambda': 0.06462161891108148, 'n_estimators': 816}. Best is trial 1 with value: 0.6447430356455133.


In [ ]:
print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)

Best Score : 0.6447430356455133
Best trial : {'max_depth': 14, 'colsample_bytree': 0.9499975771450039, 'colsample_bylevel': 0.9276266278535859, 'colsample_bynode': 0.4733477167159189, 'min_child_weight': 247.76100749471476, 'gamma': 6.095937074798688, 'subsample': 0.6652414479118475, 'reg_alpha': 0.010211256122219953, 'reg_lambda': 0.23410118458859783, 'n_estimators': 875}


In [ ]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [ ]:
# 파라미터들과의 관계
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

In [ ]:
n_fold = 5
cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 39)

xgb_val = np.zeros((X_train.shape[0], 2))
xgb_part_test = np.zeros((X_part_test.shape[0], 2))
xgb_test = np.zeros((X_test.shape[0], 2))

print(xgb_val.shape)
print(xgb_part_test.shape)
print(xgb_test.shape)

for i, (i_trn, i_val) in enumerate(cv.split(X_train, y_train), 1):
    print(f'training model for CV #{i}')
    optuna_xgb = xgb.XGBClassifier(
                        random_state = 39,
                        max_depth = 14, 
                        colsample_bytree = 0.9499975771450039, 
                        colsample_bylevel = 0.9276266278535859, 
                        colsample_bynode = 0.4733477167159189, 
                        min_child_weight = 247.76100749471476, 
                        gamma = 6.095937074798688, 
                        subsample = 0.6652414479118475, 
                        reg_alpha = 0.010211256122219953, 
                        reg_lambda = 0.23410118458859783, 
                        n_estimators = 875,
                        verbose = False)

    optuna_xgb.fit(X_train.loc[i_trn, :], y_train[i_trn], eval_set = [(X_train.loc[i_val, :], y_train[i_val])], early_stopping_rounds = 100, eval_metric = 'auc')

    xgb_val[i_val, :] = optuna_xgb.predict_proba(X_train.loc[i_val, :])
    xgb_part_test += optuna_xgb.predict_proba(X_part_test) / n_fold
    xgb_test += optuna_xgb.predict_proba(X_test) / n_fold

(456972, 2)
(44979, 2)
(46404, 2)
training model for CV #1
[0]	validation_0-auc:0.629393
Will train until validation_0-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.637822
[2]	validation_0-auc:0.642414
[3]	validation_0-auc:0.644423
[4]	validation_0-auc:0.645923
[5]	validation_0-auc:0.64635
[6]	validation_0-auc:0.647356
[7]	validation_0-auc:0.648414
[8]	validation_0-auc:0.64957
[9]	validation_0-auc:0.650202
[10]	validation_0-auc:0.650973
[11]	validation_0-auc:0.651737
[12]	validation_0-auc:0.652516
[13]	validation_0-auc:0.652796
[14]	validation_0-auc:0.653256
[15]	validation_0-auc:0.653989
[16]	validation_0-auc:0.654408
[17]	validation_0-auc:0.654899
[18]	validation_0-auc:0.655293
[19]	validation_0-auc:0.655818
[20]	validation_0-auc:0.656557
[21]	validation_0-auc:0.657235
[22]	validation_0-auc:0.65769
[23]	validation_0-auc:0.658101
[24]	validation_0-auc:0.658531
[25]	validation_0-auc:0.658872
[26]	validation_0-auc:0.659252
[27]	validation_0-auc:0.659522
[28]	validation_0-auc

In [ ]:
xgb_part_test

array([[0.39201721, 0.6079828 ],
       [0.6856394 , 0.31436063],
       [0.80329819, 0.19670181],
       ...,
       [0.36734895, 0.63265105],
       [0.44900814, 0.55099186],
       [0.52853276, 0.47146724]])

In [ ]:
print(xgb_val.shape)
print(xgb_part_test.shape)
print(xgb_test.shape)

xgb_pred = xgb_val[:, 1]
xgb_pred = np.where(xgb_pred >= 0.5, 1, 0)
print(f1_score(y_train, xgb_pred))

xgb_pred2 = xgb_part_test[:, 1]
xgb_pred2 = np.where(xgb_pred2 >= 0.5, 1, 0)
print(f1_score(y_part_test, xgb_pred2))

(456972, 2)
(44979, 2)
(46404, 2)
0.6444481484685385
0.606824512534819


In [ ]:
# 저장

xgb_val_file = pd.DataFrame(xgb_val, columns = ['0', '1'])
xgb_part_test_file = pd.DataFrame(xgb_part_test, columns = ['0', '1'])
xgb_test_file = pd.DataFrame(xgb_test, columns = ['0', '1'])
xgb_val_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/xgb_val.csv', index = False)
xgb_part_test_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/xgb_part_test.csv', index = False)
xgb_test_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/xgb_test.csv', index = False)

### 4) extratree

In [ ]:
def objective(trial : Trial) -> float :

    params_etc = {
        "random_state" : 39,
        'n_jobs' : -1,
        'max_depth' : trial.suggest_int("max_depth", 1, 20),
        'n_estimators' : trial.suggest_int("n_estimators", 100, 1000),
        'min_samples_split' : trial.suggest_int('min_samples_split', 2, 15),
        'min_samples_leaf' : trial.suggest_int('min_samples_leaf', 1, 15),
        'max_features' : trial.suggest_uniform('max_features', 0.1, 1.0)
  }
    
    model = ExtraTreesClassifier(**params_etc)
    model.fit(X_part_train, y_part_train)

    etc_pred = model.predict(X_val)
    f1 = f1_score(y_val, etc_pred)
    
    return f1

In [ ]:
sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "etc_parameter_opt",
    direction = "maximize",
    sampler = sampler)
study.optimize(objective, n_trials = 40)

[I 2022-01-17 17:03:00,759] A new study created in memory with name: etc_parameter_opt
[I 2022-01-17 17:18:41,247] Trial 0 finished with value: 0.6275219590098482 and parameters: {'max_depth': 11, 'n_estimators': 818, 'min_samples_split': 13, 'min_samples_leaf': 2, 'max_features': 0.6418018061122942}. Best is trial 0 with value: 0.6275219590098482.
[I 2022-01-17 17:32:47,790] Trial 1 finished with value: 0.6284026434900869 and parameters: {'max_depth': 11, 'n_estimators': 517, 'min_samples_split': 8, 'min_samples_leaf': 10, 'max_features': 0.9330974924253972}. Best is trial 1 with value: 0.6284026434900869.
[I 2022-01-17 18:06:32,798] Trial 2 finished with value: 0.6344836389940854 and parameters: {'max_depth': 17, 'n_estimators': 950, 'min_samples_split': 14, 'min_samples_leaf': 7, 'max_features': 0.8432307660728353}. Best is trial 2 with value: 0.6344836389940854.
[I 2022-01-17 18:29:49,826] Trial 3 finished with value: 0.633894194975644 and parameters: {'max_depth': 19, 'n_estimator

In [ ]:
print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)

Best Score : 0.6344836389940854
Best trial : {'max_depth': 17, 'n_estimators': 950, 'min_samples_split': 14, 'min_samples_leaf': 7, 'max_features': 0.8432307660728353}


In [ ]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [ ]:
# 파라미터들과의 관계
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

In [ ]:
n_fold = 5
cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 39)

etc_val = np.zeros((X_train.shape[0], 2))
etc_part_test = np.zeros((X_part_test.shape[0], 2))
etc_test = np.zeros((X_test.shape[0], 2))

print(etc_val.shape)
print(etc_part_test.shape)
print(etc_test.shape)

for i, (i_trn, i_val) in enumerate(cv.split(X_train, y_train), 1):
    print(f'training model for CV #{i}')
    optuna_etc = ExtraTreesClassifier(
                        random_state = 39,
                        min_samples_split = 14, 
                        min_samples_leaf = 7, 
                        max_features = 0.8432307660728353,
                        max_depth = 17,
                        n_estimators = 950)

    optuna_etc.fit(X_train.loc[i_trn, :], y_train[i_trn])

    etc_val[i_val, :] = optuna_etc.predict_proba(X_train.loc[i_val, :])
    etc_part_test += optuna_etc.predict_proba(X_part_test) / n_fold
    etc_test += optuna_etc.predict_proba(X_test) / n_fold

(456972, 2)
(44979, 2)
(46404, 2)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5


In [ ]:
print(etc_val.shape)
print(etc_part_test.shape)
print(etc_test.shape)

etc_pred = etc_val[:, 1]
etc_pred = np.where(etc_pred >= 0.5, 1, 0)
print(f1_score(y_train, etc_pred))

etc_pred2 = etc_part_test[:, 1]
etc_pred2 = np.where(etc_pred2 >= 0.5, 1, 0)
print(f1_score(y_part_test, etc_pred2))

(456972, 2)
(44979, 2)
(46404, 2)
0.6320895788460369
0.6213876566604936


In [ ]:
# 저장

etc_val_file = pd.DataFrame(etc_val, columns = ['0', '1'])
etc_part_test_file = pd.DataFrame(etc_part_test, columns = ['0', '1'])
etc_test_file = pd.DataFrame(etc_test, columns = ['0', '1'])
etc_val_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/etc_val.csv', index = False)
etc_part_test_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/etc_part_test.csv', index = False)
etc_test_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/etc_test.csv', index = False)

### 5) catboost

In [ ]:
tt1 = X_train.columns.tolist() 
tt2 = numerical_feats
print(len(tt1))
print(len(tt2))
tt3 = []
for i in tt1 :
  if i not in tt2 :
    tt3.append(i)
print(len(tt3))
print(tt3)

cat_cols = tt3

90
5
85
['person_attribute_a', 'person_attribute_a_under', 'person_attribute_b', 'person_prefer_c', 'person_prefer_d_1', 'person_prefer_d_2', 'person_prefer_d_3', 'person_prefer_e', 'person_prefer_h_1', 'person_prefer_h_2', 'person_prefer_h_3', 'contents_attribute_i', 'contents_attribute_a', 'contents_attribute_j_under', 'contents_attribute_j', 'contents_attribute_c', 'contents_attribute_k', 'contents_attribute_l', 'contents_attribute_d', 'contents_attribute_m', 'contents_attribute_e', 'contents_attribute_h', 'person_rn', 'contents_rn', 'person_prefer_d_1_세', 'person_prefer_d_1_소', 'person_prefer_d_1_중', 'person_prefer_d_1_대', 'person_prefer_d_2_세', 'person_prefer_d_2_소', 'person_prefer_d_2_중', 'person_prefer_d_2_대', 'person_prefer_d_3_세', 'person_prefer_d_3_소', 'person_prefer_d_3_중', 'person_prefer_d_3_대', 'contents_attribute_d_세', 'contents_attribute_d_소', 'contents_attribute_d_중', 'contents_attribute_d_대', 'person_prefer_h_1_중', 'person_prefer_h_1_대', 'person_prefer_h_2_중', 'person_

In [ ]:
temp = X_part_train.columns[X_part_train.nunique() > 2].tolist()
temp2 = set(temp)
cat_cols2 = set(cat_cols)
cat_cols3 = list(temp2 & cat_cols2)
print(len(cat_cols3))

58


In [ ]:
def objective(trial : Trial) -> float :

    params_cat = {
        "random_state" : 39,
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 5, 30.00),
        "n_estimators":trial.suggest_int("n_estimators", 100, 1000),
        "max_depth":trial.suggest_int("max_depth", 8, 16),
        'random_strength' :trial.suggest_int('random_strength', 0, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
        'od_type': 'Iter',
        'one_hot_max_size' : trial.suggest_int('one_hot_max_size', 0, 5)
  }
    
    model = CatBoostClassifier(**params_cat,  task_type = "GPU")
    model.fit(X_part_train, y_part_train, eval_set = [(X_val, y_val)],
              early_stopping_rounds = 100, cat_features = cat_cols3, verbose = False)

    cat_pred = model.predict(X_val)
    f1 = f1_score(y_val, cat_pred)
    
    return f1

In [ ]:
sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "cat_parameter_opt",
    direction = "maximize",
    sampler = sampler)
study.optimize(objective, n_trials = 20)

[I 2022-01-18 12:09:52,920] A new study created in memory with name: cat_parameter_opt
Custom logger is already specified. Specify more than one logger at same time is not thread safe.[I 2022-01-18 12:12:35,442] Trial 0 finished with value: 0.619289682628939 and parameters: {'learning_rate': 0.012410186619312995, 'bagging_temperature': 5.948283319926105, 'n_estimators': 839, 'max_depth': 9, 'random_strength': 60, 'max_bin': 358, 'one_hot_max_size': 2}. Best is trial 0 with value: 0.619289682628939.
[I 2022-01-18 12:45:08,546] Trial 1 finished with value: 0.623189887219326 and parameters: {'learning_rate': 0.008767817244598053, 'bagging_temperature': 1.5849635319969935, 'n_estimators': 934, 'max_depth': 15, 'random_strength': 95, 'max_bin': 476, 'one_hot_max_size': 2}. Best is trial 1 with value: 0.623189887219326.
[I 2022-01-18 12:48:02,875] Trial 2 finished with value: 0.6439184084613448 and parameters: {'learning_rate': 0.04483569690392285, 'bagging_temperature': 19.5101753062082, 'n

In [ ]:
print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)

Best Score : 0.6439184084613448
Best trial : {'learning_rate': 0.04483569690392285, 'bagging_temperature': 19.5101753062082, 'n_estimators': 665, 'max_depth': 10, 'random_strength': 59, 'max_bin': 458, 'one_hot_max_size': 0}


In [ ]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [ ]:
# 파라미터들과의 관계
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

In [ ]:
n_fold = 5
cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 39)

cat_val = np.zeros((X_train.shape[0], 2))
cat_part_test = np.zeros((X_part_test.shape[0], 2))
cat_test = np.zeros((X_test.shape[0], 2))

print(cat_val.shape)
print(cat_part_test.shape)
print(cat_test.shape)

for i, (i_trn, i_val) in enumerate(cv.split(X_train, y_train), 1):
    print(f'training model for CV #{i}')
    optuna_cat = CatBoostClassifier(
        random_state = 39,
        learning_rate = 0.04483569690392285,
        bagging_temperature = 19.5101753062082,
        n_estimators = 665,
        max_depth = 10,
        random_strength = 59,
        max_bin = 458,
        od_type = 'Iter',
        one_hot_max_size = 0, 
        task_type = "GPU")

    optuna_cat.fit(X_train.loc[i_trn, :], y_train[i_trn], verbose = False, cat_features = cat_cols3)

    cat_val[i_val, :] = optuna_cat.predict_proba(X_train.loc[i_val, :])
    cat_part_test += optuna_cat.predict_proba(X_part_test) / n_fold
    cat_test += optuna_cat.predict_proba(X_test) / n_fold

(456972, 2)
(44979, 2)
(46404, 2)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5


In [ ]:
print(cat_val.shape)
print(cat_part_test.shape)
print(cat_test.shape)

cat_pred = cat_val[:, 1]
cat_pred = np.where(cat_pred >= 0.5, 1, 0)
print(f1_score(y_train, cat_pred))

cat_pred2 = cat_part_test[:, 1]
cat_pred2 = np.where(cat_pred2 >= 0.5, 1, 0)
print(f1_score(y_part_test, cat_pred2))

(456972, 2)
(44979, 2)
(46404, 2)
0.6359777695070903
0.6211592254483163


In [ ]:
# 저장

cat_val_file = pd.DataFrame(cat_val, columns = ['0', '1'])
cat_part_test_file = pd.DataFrame(cat_part_test, columns = ['0', '1'])
cat_test_file = pd.DataFrame(cat_test, columns = ['0', '1'])
cat_val_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/cat_val.csv', index = False)
cat_part_test_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/cat_part_test.csv', index = False)
cat_test_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/cat_test.csv', index = False)

### 6) HGBM

In [ ]:
def objective(trial : Trial) -> float :

    params_cat = {
        "random_state" : 39,
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'max_iter': 1000,
        'max_depth': trial.suggest_int('max_depth', 2, 30),
        'max_bins': trial.suggest_int('max_bins', 100, 255),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 50),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 20, 80)
  }
    
    model = HistGradientBoostingClassifier(**params_cat)
    model.fit(X_part_train, y_part_train)

    hgbm_pred = model.predict(X_val)
    f1 = f1_score(y_val, hgbm_pred)
    
    return f1

In [ ]:
sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "hgbm_parameter_opt",
    direction = "maximize",
    sampler = sampler)
study.optimize(objective, n_trials = 50)

[I 2022-01-19 09:48:38,288] A new study created in memory with name: hgbm_parameter_opt
[I 2022-01-19 09:50:24,315] Trial 0 finished with value: 0.57797644037241 and parameters: {'l2_regularization': 0.00010370046829497107, 'learning_rate': 0.03942739125359127, 'max_depth': 25, 'max_bins': 119, 'min_samples_leaf': 31, 'max_leaf_nodes': 52}. Best is trial 0 with value: 0.57797644037241.
[I 2022-01-19 09:53:20,714] Trial 1 finished with value: 0.6368586184217618 and parameters: {'l2_regularization': 1.267629869136476e-05, 'learning_rate': 0.008767817244598053, 'max_depth': 20, 'max_bins': 244, 'min_samples_leaf': 41, 'max_leaf_nodes': 77}. Best is trial 1 with value: 0.6368586184217618.
[I 2022-01-19 09:55:42,306] Trial 2 finished with value: 0.6401209890644499 and parameters: {'l2_regularization': 1.3044774098574383, 'learning_rate': 0.006755563688908309, 'max_depth': 25, 'max_bins': 247, 'min_samples_leaf': 32, 'max_leaf_nodes': 35}. Best is trial 2 with value: 0.6401209890644499.
[I 2

In [ ]:
print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)

Best Score : 0.6412850999241082
Best trial : {'l2_regularization': 1.989449687751321e-10, 'learning_rate': 0.007985629409139418, 'max_depth': 11, 'max_bins': 254, 'min_samples_leaf': 19, 'max_leaf_nodes': 25}


In [ ]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [ ]:
# 파라미터들과의 관계
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

In [ ]:
'l2_regularization': 1.989449687751321e-10, 'learning_rate': 0.007985629409139418, 'max_depth': 11, 'max_bins': 254, 'min_samples_leaf': 19, 'max_leaf_nodes': 25

In [ ]:
n_fold = 5
cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 39)

hgbm_val = np.zeros((X_train.shape[0], 2))
hgbm_part_test = np.zeros((X_part_test.shape[0], 2))
hgbm_test = np.zeros((X_test.shape[0], 2))

print(hgbm_val.shape)
print(hgbm_part_test.shape)
print(hgbm_test.shape)

for i, (i_trn, i_val) in enumerate(cv.split(X_train, y_train), 1):
    print(f'training model for CV #{i}')
    optuna_hgbm = HistGradientBoostingClassifier(
        random_state = 39,
        learning_rate = 0.007985629409139418,
        max_depth = 11,
        max_bins = 254,
        min_samples_leaf = 19, 
        max_leaf_nodes = 25)

    optuna_hgbm.fit(X_train.loc[i_trn, :], y_train[i_trn])

    hgbm_val[i_val, :] = optuna_hgbm.predict_proba(X_train.loc[i_val, :])
    hgbm_part_test += optuna_hgbm.predict_proba(X_part_test) / n_fold
    hgbm_test += optuna_hgbm.predict_proba(X_test) / n_fold

(456972, 2)
(44979, 2)
(46404, 2)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5


In [ ]:
print(hgbm_val.shape)
print(hgbm_part_test.shape)
print(hgbm_test.shape)

hgbm_pred = hgbm_val[:, 1]
hgbm_pred = np.where(hgbm_pred >= 0.5, 1, 0)
print(f1_score(y_train, hgbm_pred))

hgbm_pred2 = hgbm_part_test[:, 1]
hgbm_pred2 = np.where(hgbm_pred2 >= 0.5, 1, 0)
print(f1_score(y_part_test, hgbm_pred2))

(456972, 2)
(44979, 2)
(46404, 2)
0.6196022902173243
0.6112955414548052


In [ ]:
# 저장

hgbm_val_file = pd.DataFrame(hgbm_val, columns = ['0', '1'])
hgbm_part_test_file = pd.DataFrame(hgbm_part_test, columns = ['0', '1'])
hgbm_test_file = pd.DataFrame(hgbm_test, columns = ['0', '1'])
hgbm_val_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/hgbm_val.csv', index = False)
hgbm_part_test_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/hgbm_part_test.csv', index = False)
hgbm_test_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/hgbm_test.csv', index = False)

# 5. 2차 모델 생성 (lightgbm)

In [ ]:
path = Path('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/')

model_val = ['rf_val', 'lgbm_val', 'xgb_val', 'etc_val', 'cat_val', 'hgbm_val', 'tab_val']
model_part_test = ['rf_part_test', 'lgbm_part_test', 'xgb_part_test', 'etc_part_test', 'cat_part_test', 'hgbm_part_test', 'tab_part_test']
model_test = ['rf_test', 'lgbm_test', 'xgb_test', 'etc_test', 'cat_test', 'hgbm_test', 'tab_test']

stk_train = []
stk_part_test = []
stk_test = []
feature_names1 = []
feature_names2 = []
feature_names3 = []
for model in model_val :
    stk_train.append(np.loadtxt(path / f'{model}.csv', delimiter = ','))
    feature_names1 += [f'{model}_target0', f'{model}_target1']

for model in model_part_test :
    stk_part_test.append(np.loadtxt(path / f'{model}.csv', delimiter = ','))
    feature_names2 += [f'{model}_target0', f'{model}_target1']

for model in model_test :
    stk_test.append(np.loadtxt(path / f'{model}.csv', delimiter = ','))
    feature_names3 += [f'{model}_target0', f'{model}_target1']
    
stk_train = np.hstack(stk_train)
stk_part_test = np.hstack(stk_part_test)
stk_test = np.hstack(stk_test)
print(feature_names1)
print(feature_names2)
print(feature_names3)
print('456972 =', stk_train.shape)
print('44979 =', stk_part_test.shape)
print('46404 =', stk_test.shape)
stk_train = stk_train[1 :]
stk_part_test = stk_part_test[1 :]
stk_test = stk_test[1 :]
print('456972 =', stk_train.shape)
print('44979 =', stk_part_test.shape)
print('46404 =', stk_test.shape)

['rf_val_target0', 'rf_val_target1', 'lgbm_val_target0', 'lgbm_val_target1', 'xgb_val_target0', 'xgb_val_target1', 'etc_val_target0', 'etc_val_target1', 'cat_val_target0', 'cat_val_target1', 'hgbm_val_target0', 'hgbm_val_target1', 'tab_val_target0', 'tab_val_target1']
['rf_part_test_target0', 'rf_part_test_target1', 'lgbm_part_test_target0', 'lgbm_part_test_target1', 'xgb_part_test_target0', 'xgb_part_test_target1', 'etc_part_test_target0', 'etc_part_test_target1', 'cat_part_test_target0', 'cat_part_test_target1', 'hgbm_part_test_target0', 'hgbm_part_test_target1', 'tab_part_test_target0', 'tab_part_test_target1']
['rf_test_target0', 'rf_test_target1', 'lgbm_test_target0', 'lgbm_test_target1', 'xgb_test_target0', 'xgb_test_target1', 'etc_test_target0', 'etc_test_target1', 'cat_test_target0', 'cat_test_target1', 'hgbm_test_target0', 'hgbm_test_target1', 'tab_test_target0', 'tab_test_target1']
456972 = (456973, 14)
44979 = (44980, 14)
46404 = (46405, 14)
456972 = (456972, 14)
44979 = (44

In [ ]:
print(y_train.shape)
print(y_part_test.shape)

(456972,)
(44979,)


In [ ]:
def objective(trial : Trial) -> float :

    params_lgb = {
        "random_state": 39,
        "verbosity": -1,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "n_estimators": trial.suggest_int('n_estimators', 100, 1000),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10),
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "num_leaves": trial.suggest_int("num_leaves", 2, 40),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        "subsample_freq": trial.suggest_int("subsample_freq", 1, 10),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500)
    }
    
    model = LGBMClassifier(**params_lgb)
    model.fit(stk_train, y_train, eval_set = [(stk_train, y_train), (stk_part_test, y_part_test)],
              early_stopping_rounds = 100, verbose = False)

    stk_lgbm_pred = model.predict(stk_part_test)
    f1 = f1_score(y_part_test, stk_lgbm_pred)
    
    return f1

In [ ]:
sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "stk_lgbm_parameter_opt",
    direction = "maximize",
    sampler = sampler)
study.optimize(objective, n_trials = 500)

[I 2022-01-19 12:17:14,686] A new study created in memory with name: stk_lgbm_parameter_opt
[I 2022-01-19 12:17:22,311] Trial 0 finished with value: 0.6324512045014946 and parameters: {'learning_rate': 0.16859785528215288, 'n_estimators': 818, 'reg_alpha': 8.204018800779794, 'reg_lambda': 1.2204986770815125, 'max_depth': 13, 'num_leaves': 136, 'colsample_bytree': 0.6783450434860411, 'subsample': 0.630012021097161, 'subsample_freq': 7, 'min_child_samples': 93, 'max_bin': 445}. Best is trial 0 with value: 0.6324512045014946.
[I 2022-01-19 12:17:28,745] Trial 1 finished with value: 0.6293998592045054 and parameters: {'learning_rate': 0.2838881081911679, 'n_estimators': 928, 'reg_alpha': 4.148307969361902, 'reg_lambda': 8.258119624773384, 'max_depth': 20, 'num_leaves': 162, 'colsample_bytree': 0.5513618754177441, 'subsample': 0.7146456319775294, 'subsample_freq': 9, 'min_child_samples': 9, 'max_bin': 457}. Best is trial 0 with value: 0.6324512045014946.
[I 2022-01-19 12:17:33,508] Trial 2 

In [ ]:
# Best Score : 0.6286938469013649
# Best trial : {'n_estimators': 729, 
# 'reg_alpha': 0.17560832960220718, 
# 'reg_lambda': 6.7752961532908795, 
# 'max_depth': 4, 
# 'num_leaves': 58, 
# 'colsample_bytree': 0.8287123499658561, 
# 'subsample': 0.4570695128556688, 
# 'subsample_freq': 7, 
# 'min_child_samples': 14, 
# 'max_bin': 207}

In [ ]:
print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)

Best Score : 0.6431947771159675
Best trial : {'learning_rate': 0.23906045066349726, 'n_estimators': 123, 'reg_alpha': 1.689146494326486, 'reg_lambda': 7.293859012715027, 'max_depth': 10, 'num_leaves': 2, 'colsample_bytree': 0.44842425809846037, 'subsample': 0.512172984530381, 'subsample_freq': 7, 'min_child_samples': 36, 'max_bin': 234}


In [ ]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [ ]:
# 파라미터들과의 관계
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

In [ ]:
'learning_rate': 0.23906045066349726, 
'n_estimators': 123, 
'reg_alpha': 1.689146494326486, 
'reg_lambda': 7.293859012715027, 
'max_depth': 10, 
'num_leaves': 2, 
'colsample_bytree': 0.44842425809846037, 
'subsample': 0.512172984530381, 
'subsample_freq': 7, 
'min_child_samples': 36, 
'max_bin': 234

In [ ]:
n_fold = 5
cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 39)

stk_lgbm_val2 = np.zeros((stk_train.shape[0], 2))
stk_lgbm_part_test2 = np.zeros((stk_part_test.shape[0], 2))
stk_lgbm_test2 = np.zeros((stk_test.shape[0], 2))

print(stk_lgbm_val2.shape)
print(stk_lgbm_part_test2.shape)
print(stk_lgbm_test2.shape)

for i, (i_trn, i_val) in enumerate(cv.split(stk_train, y_train), 1):
    print(f'training model for CV #{i}')
    optuna_stk_lgbm2 = LGBMClassifier(
                        random_state = 39,
                        n_estimators = 123, 
                        reg_alpha = 1.689146494326486, 
                        reg_lambda = 7.293859012715027, 
                        max_depth = 10, 
                        num_leaves = 2, 
                        colsample_bytree = 0.44842425809846037, 
                        subsample = 0.512172984530381, 
                        subsample_freq = 7, 
                        min_child_samples = 36, 
                        max_bin = 234,
                        verbose = 100)

    optuna_stk_lgbm2.fit(stk_train[i_trn], y_train[i_trn], eval_set = [(stk_train[i_val], y_train[i_val])], early_stopping_rounds = 100, eval_metric = 'auc')

    stk_lgbm_val2[i_val, :] = optuna_stk_lgbm2.predict_proba(stk_train[i_val])
    stk_lgbm_part_test2 += optuna_stk_lgbm2.predict_proba(stk_part_test) / n_fold
    stk_lgbm_test2 += optuna_stk_lgbm2.predict_proba(stk_test) / n_fold

(456972, 2)
(44979, 2)
(46404, 2)
training model for CV #1
[1]	valid_0's auc: 0.628988	valid_0's binary_logloss: 0.686022
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.646035	valid_0's binary_logloss: 0.680148
[3]	valid_0's auc: 0.659895	valid_0's binary_logloss: 0.675159
[4]	valid_0's auc: 0.662629	valid_0's binary_logloss: 0.670758
[5]	valid_0's auc: 0.666346	valid_0's binary_logloss: 0.667074
[6]	valid_0's auc: 0.670766	valid_0's binary_logloss: 0.66364
[7]	valid_0's auc: 0.67475	valid_0's binary_logloss: 0.660809
[8]	valid_0's auc: 0.676116	valid_0's binary_logloss: 0.658102
[9]	valid_0's auc: 0.677184	valid_0's binary_logloss: 0.656031
[10]	valid_0's auc: 0.679777	valid_0's binary_logloss: 0.653975
[11]	valid_0's auc: 0.680165	valid_0's binary_logloss: 0.651985
[12]	valid_0's auc: 0.681373	valid_0's binary_logloss: 0.650276
[13]	valid_0's auc: 0.681771	valid_0's binary_logloss: 0.648733
[14]	valid_0's auc: 0.683668	valid_0's binary_logloss: 0.

In [ ]:
print(stk_lgbm_val2.shape)
print(stk_lgbm_part_test2.shape)
print(stk_lgbm_test2.shape)

(456972, 2)
(44979, 2)
(46404, 2)


In [ ]:
scores = np.zeros((100, 2))

for i in range(0, 100) :
    threshold = 0.3 + i / 1000
    pred1 = stk_lgbm_val2[:, 1]
    pred1 = np.where(pred1 >= threshold, 1, 0)
    score = f1_score(y_train, pred1)
    scores[i, 0] = threshold
    scores[i, 1] = score

print(max(scores[:, 1]))


0.6985602296121941


In [ ]:
scores[scores[:, 1] == 0.6985602296121941] # 0.355

array([[0.355     , 0.69856023]])

In [ ]:
scores = np.zeros((100, 2))

for i in range(0, 100) :
    threshold = 0.3 + i / 1000
    pred2 = stk_lgbm_part_test2[:, 1]
    pred2 = np.where(pred2 >= threshold, 1, 0)
    score = f1_score(y_part_test, pred2)
    scores[i, 0] = threshold
    scores[i, 1] = score

print(max(scores[:, 1]))


0.6853723312523712


In [ ]:
scores[scores[:, 1] == 0.6853723312523712] # 0.353

array([[0.353     , 0.68537233]])

In [ ]:
threshold = 0.353

pred_list = []
scores = []

pred3 = stk_lgbm_test2[:, 1]
pred3 = np.where(pred3 >= threshold , 1, 0)

In [ ]:
# 제출

submission_preds = pred3
submission = pd.read_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/sample_submission.csv')
submission['target'] = submission_preds
submission.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/submission_stk_lgbm_wj2.csv', index = False)

# 6. 2차 모델 생성 (catboost)

In [ ]:
path = Path('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/')

model_val = ['rf_val', 'lgbm_val', 'xgb_val', 'etc_val', 'cat_val', 'hgbm_val', 'tab_val']
model_part_test = ['rf_part_test', 'lgbm_part_test', 'xgb_part_test', 'etc_part_test', 'cat_part_test', 'hgbm_part_test', 'tab_part_test']
model_test = ['rf_test', 'lgbm_test', 'xgb_test', 'etc_test', 'cat_test', 'hgbm_test', 'tab_test']

stk_train = []
stk_part_test = []
stk_test = []
feature_names1 = []
feature_names2 = []
feature_names3 = []
for model in model_val :
    stk_train.append(np.loadtxt(path / f'{model}.csv', delimiter = ','))
    feature_names1 += [f'{model}_target0', f'{model}_target1']

for model in model_part_test :
    stk_part_test.append(np.loadtxt(path / f'{model}.csv', delimiter = ','))
    feature_names2 += [f'{model}_target0', f'{model}_target1']

for model in model_test :
    stk_test.append(np.loadtxt(path / f'{model}.csv', delimiter = ','))
    feature_names3 += [f'{model}_target0', f'{model}_target1']
    
stk_train = np.hstack(stk_train)
stk_part_test = np.hstack(stk_part_test)
stk_test = np.hstack(stk_test)
print(feature_names1)
print(feature_names2)
print(feature_names3)
print('456972 =', stk_train.shape)
print('44979 =', stk_part_test.shape)
print('46404 =', stk_test.shape)
stk_train = stk_train[1 :]
stk_part_test = stk_part_test[1 :]
stk_test = stk_test[1 :]
print('456972 =', stk_train.shape)
print('44979 =', stk_part_test.shape)
print('46404 =', stk_test.shape)

['rf_val_target0', 'rf_val_target1', 'lgbm_val_target0', 'lgbm_val_target1', 'xgb_val_target0', 'xgb_val_target1', 'etc_val_target0', 'etc_val_target1', 'cat_val_target0', 'cat_val_target1', 'hgbm_val_target0', 'hgbm_val_target1', 'tab_val_target0', 'tab_val_target1']
['rf_part_test_target0', 'rf_part_test_target1', 'lgbm_part_test_target0', 'lgbm_part_test_target1', 'xgb_part_test_target0', 'xgb_part_test_target1', 'etc_part_test_target0', 'etc_part_test_target1', 'cat_part_test_target0', 'cat_part_test_target1', 'hgbm_part_test_target0', 'hgbm_part_test_target1', 'tab_part_test_target0', 'tab_part_test_target1']
['rf_test_target0', 'rf_test_target1', 'lgbm_test_target0', 'lgbm_test_target1', 'xgb_test_target0', 'xgb_test_target1', 'etc_test_target0', 'etc_test_target1', 'cat_test_target0', 'cat_test_target1', 'hgbm_test_target0', 'hgbm_test_target1', 'tab_test_target0', 'tab_test_target1']
456972 = (456973, 14)
44979 = (44980, 14)
46404 = (46405, 14)
456972 = (456972, 14)
44979 = (44

In [ ]:
print(y_train.shape)
print(y_part_test.shape)

(456972,)
(44979,)


In [ ]:
def objective(trial : Trial) -> float :

    params_cat = {
        "random_state" : 39,
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 0.01),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 5, 30.00),
        "n_estimators":trial.suggest_int("n_estimators", 100, 1000),
        "max_depth":trial.suggest_int("max_depth", 8, 16),
        'random_strength' :trial.suggest_int('random_strength', 0, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
        'od_type': 'Iter',
        'one_hot_max_size' : trial.suggest_int('one_hot_max_size', 0, 5)
  }
    
    model = CatBoostClassifier(**params_cat,  task_type = "GPU")
    model.fit(stk_train, y_train, eval_set = [(stk_part_test, y_part_test)],
              early_stopping_rounds = 100, verbose = False)

    stk_cat_pred = model.predict(stk_part_test)
    f1 = f1_score(y_part_test, stk_cat_pred)
    
    return f1

In [ ]:
sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "stk_cat_parameter_opt",
    direction = "maximize",
    sampler = sampler)
study.optimize(objective, n_trials = 50)

[I 2022-01-19 13:28:29,743] A new study created in memory with name: stk_cat_parameter_opt
Custom logger is already specified. Specify more than one logger at same time is not thread safe.[I 2022-01-19 13:28:46,988] Trial 0 finished with value: 0.6320800245689466 and parameters: {'learning_rate': 0.012410186619312995, 'bagging_temperature': 20.886040962658566, 'n_estimators': 839, 'max_depth': 9, 'random_strength': 60, 'max_bin': 358, 'one_hot_max_size': 2}. Best is trial 0 with value: 0.6320800245689466.
[I 2022-01-19 13:34:33,626] Trial 1 finished with value: 0.6314122305323362 and parameters: {'learning_rate': 0.008767817244598053, 'bagging_temperature': 15.535180162253402, 'n_estimators': 934, 'max_depth': 15, 'random_strength': 95, 'max_bin': 476, 'one_hot_max_size': 2}. Best is trial 0 with value: 0.6320800245689466.
[I 2022-01-19 13:34:44,907] Trial 2 finished with value: 0.6318373071528752 and parameters: {'learning_rate': 0.04483569690392285, 'bagging_temperature': 27.24604684

In [ ]:
print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)

Best Score : 0.633095102380848
Best trial : {'learning_rate': 0.0317622931448512, 'bagging_temperature': 14.85441935429381, 'n_estimators': 775, 'max_depth': 9, 'random_strength': 13, 'max_bin': 382, 'one_hot_max_size': 2}


In [ ]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [ ]:
# 파라미터들과의 관계
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

In [ ]:
'learning_rate': 0.0317622931448512, 
'bagging_temperature': 14.85441935429381, 
'n_estimators': 775, 
'max_depth': 9, 
'random_strength': 13, 
'max_bin': 382, 
'one_hot_max_size': 2

In [ ]:
n_fold = 5
cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 39)

stk_cat_val = np.zeros((stk_train.shape[0], 2))
stk_cat_part_test = np.zeros((stk_part_test.shape[0], 2))
stk_cat_test = np.zeros((stk_test.shape[0], 2))

print(stk_cat_val.shape)
print(stk_cat_part_test.shape)
print(stk_cat_test.shape)

for i, (i_trn, i_val) in enumerate(cv.split(stk_train, y_train), 1):
    print(f'training model for CV #{i}')
    optuna_stk_cat = CatBoostClassifier(
                        random_state = 39,
                        learning_rate = 0.0317622931448512, 
                        bagging_temperature = 14.85441935429381, 
                        n_estimators = 775, 
                        max_depth = 9, 
                        random_strength = 13, 
                        max_bin = 382, 
                        one_hot_max_size = 2,
                        task_type = "GPU")

    optuna_stk_cat.fit(stk_train[i_trn], y_train[i_trn], eval_set = [(stk_train[i_val], y_train[i_val])], verbose = False)

    stk_cat_val[i_val, :] = optuna_stk_cat.predict_proba(stk_train[i_val])
    stk_cat_part_test += optuna_stk_cat.predict_proba(stk_part_test) / n_fold
    stk_cat_test += optuna_stk_cat.predict_proba(stk_test) / n_fold

(456972, 2)
(44979, 2)
(46404, 2)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5


In [ ]:
print(stk_cat_val.shape)
print(stk_cat_part_test.shape)
print(stk_cat_test.shape)

(456972, 2)
(44979, 2)
(46404, 2)


In [ ]:
scores = np.zeros((100, 2))

for i in range(0, 100) :
    threshold = 0.3 + i / 1000
    pred1 = stk_cat_val[:, 1]
    pred1 = np.where(pred1 >= threshold, 1, 0)
    score = f1_score(y_train, pred1)
    scores[i, 0] = threshold
    scores[i, 1] = score

print(max(scores[:, 1]))


0.6990907793439843


In [ ]:
scores[scores[:, 1] == 0.6990907793439843] # 0.349

array([[0.349     , 0.69909078]])

In [ ]:
scores = np.zeros((100, 2))

for i in range(0, 100) :
    threshold = 0.3 + i / 1000
    pred2 = stk_cat_part_test[:, 1]
    pred2 = np.where(pred2 >= threshold, 1, 0)
    score = f1_score(y_part_test, pred2)
    scores[i, 0] = threshold
    scores[i, 1] = score

print(max(scores[:, 1]))


0.6860532704208978


In [ ]:
scores[scores[:, 1] == 0.6860532704208978] # 0.336

array([[0.336     , 0.68605327]])

In [ ]:
threshold = 0.336

pred3 = stk_cat_test[:, 1]
pred3 = np.where(pred3 >= threshold , 1, 0)

In [ ]:
# 제출

submission_preds = pred3
submission = pd.read_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/sample_submission.csv')
submission['target'] = submission_preds
submission.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/submission_stk_cat_wj.csv', index = False)

In [ ]:
threshold = 0.349

pred_list = []
scores = []

pred4 = stk_cat_test[:, 1]
pred4 = np.where(pred4 >= threshold , 1, 0)

# 제출

submission_preds = pred4
submission = pd.read_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/sample_submission.csv')
submission['target'] = submission_preds
submission.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/submission_stk_cat_wj2.csv', index = False)